In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
df = pd.read_csv("data/bank_reviews_clean.csv")



In [7]:
# Basic preprocessing
df['review'] = df['review'].str.lower().fillna("")

vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words='english', max_features=100)
X = vectorizer.fit_transform(df['review'])
features = vectorizer.get_feature_names_out()

In [8]:
# Extract top N keywords per bank
bank_keywords = {}
for bank in df['bank'].unique():
    texts = df[df['bank'] == bank]['review']
    vec = TfidfVectorizer(ngram_range=(1, 2), stop_words='english', max_features=100)
    X_bank = vec.fit_transform(texts)
    keywords = vec.get_feature_names_out()
    bank_keywords[bank] = keywords[:10]  # Top 10 keywords

# Save top keywords
for bank, keywords in bank_keywords.items():
    print(f"{bank}: {keywords}")

CBE: ['2025' 'access' 'account' 'add' 'amazing' 'app' 'application' 'apps'
 'bad' 'bank']
BOA: ['account' 'amazing' 'android' 'app' 'application' 'apps' 'available'
 'awesome' 'bad' 'bank']
Dashen: ['able' 'account' 'ahead' 'amazing' 'amazing app' 'app' 'application'
 'bank' 'bank super' 'banking']


In [9]:
def assign_theme(text):
    text = str(text).lower()
    if any(word in text for word in ['login', 'sign', 'password']):
        return "Account Access Issues"
    elif any(word in text for word in ['transfer', 'delay', 'money', 'failed']):
        return "Transaction Problems"
    elif any(word in text for word in ['ui', 'interface', 'navigation', 'design']):
        return "UI/UX"
    else:
        return "Other"

df['theme'] = df['review'].apply(assign_theme)
df.to_csv("thematic_analysis.csv", index=False)